In [69]:
import sys
from pprint import pprint
import pybmd

from python_get_resolve import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
sub_folder = media_pool.GetRootFolder().GetSubFolderList()[0]

# # create timeline 1
# project.SetSetting('timelineResolutionWidth', "1920")
# project.SetSetting('timelineResolutionHeight', "1080")
# media_pool.CreateEmptyTimeline("timeline 1")
# # while True:
# #     terminate_input = input("请取消勾选 timeline 1 的 use project setting，取消勾选完毕请输入 OK。程序将继续运行。")
# #     if terminate_input == "OK":
# #         break
# media_pool.AppendToTimeline(sub_folder.GetClipList()[0])
# project.GetCurrentTimeline().SetSetting("timelineResolutionWidth", "3840")
# project.GetCurrentTimeline().SetSetting("timelineResolutionHeight", "2160")
# if project.SetSetting('timelineFrameRate', float(25)):
#     print("set timeline frame rate successful")
# print(f"timeline 1 width: {project.GetCurrentTimeline().GetSetting()['timelineResolutionWidth']}")
# print(f"timeline 1 height: {project.GetCurrentTimeline().GetSetting()['timelineResolutionHeight']}")
# print(f"timeline 1 frame rate: {project.GetCurrentTimeline().GetSetting()['timelineFrameRate']}")

root_folder = media_pool.GetRootFolder()
for sub_folder in root_folder.GetSubFolderList():
    for clip in sub_folder.GetClipList():
        print(clip.GetName())


A008C0081_220620_005G.MOV
A008C0076_220620_005G.MOV
A008C0101_220620_005G.MOV
A010C0044_220622_005G.MOV
A010C0037_220622_005G.MOV
A011C0022_220623_005G.MOV
A011C0033_220623_005G.MOV
A011C0010_220623_005G.MOV
A011C0018_220623_005G.MOV
A011C0041_220623_005G.MOV
A011C0051_220623_005G.MOV
A011C0034_220623_005G.MOV
220709_FX3_A_0083.MP4
A011C0093_220624_005G.MOV
A011C0096_220624_005G.MOV
A011C0035_220623_005G.MOV
FX6#3_2048x856
Ronin_4D#1_2048x856
Ronin_4D#1_2048x1080
Ronin_4D#2_2048x1080
Ronin_4D#2_1920x1080
航拍#4_2048x1080


In [ ]:
from pybmd import Bmd

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()

clip_width = 4096

media_pool.create_empty_timeline("tmp")
timeline = project.get_current_timeline()
print(f"1_current_timeline: {timeline.get_name()}")

media_pool.create_empty_timeline("tmp1")
timeline = project.get_current_timeline()
print(f"2_current_timeline: {timeline.get_name()}")

media_pool.create_empty_timeline("tmp")
timeline_number = project.get_timeline_count()
for i in range(timeline_number):
    timeline = project.get_timeline_by_index(i + 1)
    # print(timeline.get_setting()["timelineResolutionWidth"])
    if timeline.get_setting()["timelineResolutionWidth"] == str(clip_width):
        while project.set_current_timeline(timeline):
            print("successful")
            break
timeline = project.get_current_timeline()
print(f"3_current_timeline: {timeline.get_name()}")


`createEmptyTimeline()` 在遇到名称与之前建立的时间线重复之时，会直接跳过，不采取什么动作。通常情况下 `createEmptyTimeline()` 之后会把 currentTimeline 设为新建的那条时间线，但是遇到重复的话。

What I need to do now: 找到目前新建的时间线当中，分辨率 width 与当前这个 clip 相符的那条时间线，把它设为 current timeline。

In [94]:
from pybmd import Bmd
from pprint import pprint

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()
root_folder = media_pool.get_root_folder()

for sub_folder in root_folder.get_sub_folder_list():
    if sub_folder.get_name() == "Ronin_4D#2":
        for index, clip in enumerate(sub_folder.get_clip_list()):
            pprint(f"{index}_{clip.get_name()}")

'0_A011C0022_220623_005G.MOV'
'1_A011C0010_220623_005G.MOV'
'2_A011C0033_220623_005G.MOV'
'3_A011C0018_220623_005G.MOV'
'4_A011C0041_220623_005G.MOV'
'5_A011C0051_220623_005G.MOV'
'6_A011C0034_220623_005G.MOV'
'7_A011C0093_220624_005G.MOV'
'8_220709_FX3_A_0083.MP4'
'9_A011C0096_220624_005G.MOV'


In [93]:
from pybmd import Bmd

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()
root_folder = media_pool.get_root_folder()

media_storage = resolve.get_media_stroage()
# for media_storage_sub_folder in media_storage.get_sub_folder_list("/Volumes/DIT-2T#9/Users/thom/Movies/Videos/Footages/悠长假期"):
#     print(media_storage_sub_folder)
media_storage.get_sub_folder_list("/Volumes/DIT-2T#9/Users/thom/Movies/Videos/Footages/悠长假期")
media_storage.add_item_list_to_meida_pool()

['/Volumes/DIT-2T#9/Users/thom/Movies/Videos/Footages/悠长假期/FX6#3',
 '/Volumes/DIT-2T#9/Users/thom/Movies/Videos/Footages/悠长假期/Ronin_4D#1',
 '/Volumes/DIT-2T#9/Users/thom/Movies/Videos/Footages/悠长假期/Ronin_4D#2',
 '/Volumes/DIT-2T#9/Users/thom/Movies/Videos/Footages/悠长假期/航拍#4']

In [120]:
from pybmd import Bmd

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
media_pool = project.get_media_pool()

# 拿到机位 bin 下所有 clip 的分辨率信息，create new empty timeline.
for index, sub_folder in enumerate(media_pool.get_root_folder().get_sub_folder_list()):

    # 排除 _Timeline Bin
    if sub_folder.get_name() == "_Timeline":
        break

    # 拿到机位 bin 下所有 clip 的分辨率信息
    all_clips_resolution = []  # Camera 机位 bin 下所有 clip 的 resolution 信息
    for clips in sub_folder.get_clip_list():
        all_clips_resolution.append(clips.get_clip_property()["Resolution"])
    all_clips_resolution = list(dict.fromkeys(all_clips_resolution))  # 移除重复项
    print(f"{index + 1}: {sub_folder.get_name()}: {all_clips_resolution}")

    for res in all_clips_resolution:
        timeline_name = f"{sub_folder.get_name()}_{res}"
        print(res)
        media_pool.create_empty_timeline(timeline_name)

    # print(timeline_name)

1: FX6#3: ['4096x1712']
4096x1712
2: Ronin_4D#1: ['4096x1712', '2048x1080', '4096x2160']
4096x1712
2048x1080
4096x2160
3: Ronin_4D#2: ['4096x2160', '3840x2160']
4096x2160
3840x2160
4: 航拍#4: ['4096x2160']
4096x2160


In [150]:
from python_get_resolve import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()

def create_new_timeline(timeline_name: str, width: str, height: str) -> None:
    media_pool.SetCurrentFolder(root_folder.GetSubFolderList()[-1])
    media_pool.CreateEmptyTimeline(timeline_name)
    current_timeline = project.GetCurrentTimeline()
    current_timeline.SetSetting("useCustomSettings", "1")
    current_timeline.SetSetting("timelineResolutionWidth", width)
    current_timeline.SetSetting("timelineResolutionHeight", height)
    current_timeline.SetSetting("timelineFrameRate", str(float(25)))

create_new_timeline("12", "1080", "1080")